In [1]:
from pathlib import Path

# Define the PDF you're testing
pdf_path = Path(
    "/n/hausmann_lab/lab/kdaryanani/deeplearn/gl_deep_search/backend/etl/experiments/ocr_pipeline/downloaded_papers/gl_url_2e0785e3e837524b/atlas_2013_part1.pdf"
)

assert pdf_path.exists(), "PDF file does not exist!"
print(f"Testing with: {pdf_path.name}")

Testing with: atlas_2013_part1.pdf


In [3]:
from marker.converters.pdf import PdfConverter
from marker.models import create_model_dict
from marker.output import text_from_rendered

converter = PdfConverter(
    artifact_dict=create_model_dict(),
)

rendered = converter(str(pdf_path))
text, _, images = text_from_rendered(rendered)

Loaded layout model s3://layout/2025_02_18 on device cpu with dtype torch.float32


Loaded texify model s3://texify/2025_02_18 on device cpu with dtype torch.float32


Loaded recognition model s3://text_recognition/2025_02_18 on device cpu with dtype torch.float32


Loaded table recognition model s3://table_recognition/2025_02_18 on device cpu with dtype torch.float32


Loaded detection model s3://text_detection/2025_02_28 on device cpu with dtype torch.float32


Loaded detection model s3://inline_math_detection/2025_02_24 on device cpu with dtype torch.float32


Running OCR Error Detection: 100%|██████████| 18/18 [00:32<00:00,  1.79s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Texify inference: 100%|██████████| 12/12 [00:47<00:00,  3.97s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables: 100%|██████████| 3/3 [01:03<00:00, 21.31s/it]


In [4]:
text

'![](_page_0_Picture_0.jpeg)\n\n#### THE ATLAS OF ECONOMIC COMPLEXITY\n\nMAPPING PATHS TO PROSPERITY\n\n#### A U T H O R S :\n\n*Ricardo Hausmann* | *César A. Hidalgo* | *Sebastián Bustos Michele Coscia* | *Alexander Simoes* | *Muhammed A. Yıldırım*\n\n#### ACKNOWLEDGMENTS\n\nThe research on which this Atlas is based began around 2006 with the idea of the product space. In the original paper published in Science in 2007, we collaborated with Albert-Laszlo Barabasi and Bailey Klinger. The view of economic development of countries as a process of discovering which products a country can master, a process we called self-discovery, came from joint work with Dani Rodrik and later also with Jason Hwang. We explored different implications of the basic approach in papers with Dany Bahar, Bailey Klinger, Robert Lawrence, Francisco Rodriguez, Dani Rodrik, Charles Sabel, Rodrigo Wagner and Andrés Zahler. Throughout, we received significant feedback and advice from Lant Pritchett, Andrés Velasco a

In [1]:
# %%
import sys
from pathlib import Path

# Add the module path
sys.path.append(
    "/n/hausmann_lab/lab/kdaryanani/deeplearn/gl_deep_search/backend/etl/experiments/ocr_pipeline"
)

from pdf_modules import (  # or specific functions like parse_marker
    PARSERS,
    is_pdf_text_based,
)

# Define the PDF you're testing
pdf_path = Path("downloaded_papers/gl_url_2e0785e3e837524b/atlas_2013_part1.pdf")
assert pdf_path.exists(), f"PDF does not exist: {pdf_path}"
print(f"✅ Loaded: {pdf_path.name}")

# %%
# Optional: check if the PDF is text-based
result = is_pdf_text_based(pdf_path)
print(result)

# %%
# Choose a parser: 'marker', 'unstructured', 'llamaparse', or 'mistral'
parser_name = "marker"  # or "mistral", "llamaparse", etc.

parser_func = PARSERS[parser_name]
parsed_output = parser_func(pdf_path)
print(parsed_output["text"][:1000])  # Preview first 1000 characters

✅ Loaded: atlas_2013_part1.pdf
{'path': 'downloaded_papers/gl_url_2e0785e3e837524b/atlas_2013_part1.pdf', 'text_based': True, 'error': None}
Loaded layout model s3://layout/2025_02_18 on device cpu with dtype torch.float32
Loaded texify model s3://texify/2025_02_18 on device cpu with dtype torch.float32
Loaded recognition model s3://text_recognition/2025_02_18 on device cpu with dtype torch.float32
Loaded table recognition model s3://table_recognition/2025_02_18 on device cpu with dtype torch.float32
Loaded detection model s3://text_detection/2025_02_28 on device cpu with dtype torch.float32
Loaded detection model s3://inline_math_detection/2025_02_24 on device cpu with dtype torch.float32


Running OCR Error Detection: 100%|██████████| 18/18 [00:32<00:00,  1.79s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Texify inference: 100%|██████████| 12/12 [00:52<00:00,  4.34s/it]
Detecting bboxes: 0it [00:00, ?it/s]
Recognizing tables:   0%|          | 0/3 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [4]:
# %%
import sys
from pathlib import Path

sys.path.append(
    "/n/hausmann_lab/lab/kdaryanani/deeplearn/gl_deep_search/backend/etl/experiments/ocr_pipeline"
)

from pdf_modules import PARSERS

# %%
# Load your test PDF
pdf_path = Path("downloaded_papers/gl_url_2e0785e3e837524b/atlas_2013_part1.pdf")
assert pdf_path.exists()

# %%
# Run Mistral parser
parsed = PARSERS["mistral"](pdf_path)
print(parsed["text"][:1000])  # Show the first 1000 characters of output

INFO:httpx:HTTP Request: POST https://api.mistral.ai/v1/ocr "HTTP/1.1 500 Internal Server Error"


SDKError: API error occurred: Status 500
{"object":"error","message":"Service unavailable.","type":"internal_server_error","param":null,"code":"1000"}

In [ ]:
# Let's create wrapper functions for the four tools: Llamaparse, Mistral OCR API, Unstructured, and Marker.
# These wrappers will each define a `parse_<tool>` function that takes a file path and returns structured output.


def parse_marker(pdf_path):
    from marker.converters.pdf import PdfConverter
    from marker.models import create_model_dict
    from marker.output import text_from_rendered

    converter = PdfConverter(artifact_dict=create_model_dict())
    rendered = converter(str(pdf_path))
    text, _, _ = text_from_rendered(rendered)
    return text


def parse_unstructured(pdf_path):
    from unstructured.partition.pdf import partition_pdf

    elements = partition_pdf(filename=str(pdf_path))
    return [el.text for el in elements if hasattr(el, "text")]


def parse_llamaparse(pdf_path):
    import requests

    # Replace this with your actual Llamaparse endpoint and API key
    API_URL = "https://api.llamaparse.com/v1/parse"
    API_KEY = "YOUR_API_KEY"

    with open(pdf_path, "rb") as f:
        files = {"file": f}
        headers = {"Authorization": f"Bearer {API_KEY}"}
        response = requests.post(API_URL, headers=headers, files=files)

    response.raise_for_status()
    return response.json()


def parse_mistral(pdf_path):
    import requests

    # Replace this with your actual Mistral OCR endpoint and API key
    API_URL = "https://api.mistral.ai/ocr"
    API_KEY = "YOUR_API_KEY"

    with open(pdf_path, "rb") as f:
        files = {"file": f}
        headers = {"Authorization": f"Bearer {API_KEY}"}
        response = requests.post(API_URL, headers=headers, files=files)

    response.raise_for_status()
    return response.json()


# Now export the code block so you can easily reuse or plug it into a benchmarking notebook or script.